This script adapts the river depth and width files

In [ ]:
import xarray as xr
import numpy as np
import hydromt
from os.path import join, basename, isdir
import glob
import matplotlib.pyplot as plt
from rasterio.transform import from_bounds
import geopandas as gpd

In [ ]:
mdir = r'../../3_models/CMF/03min'

# from params.txt:
shape = 45, 35  # nrow, ncol 6 min
shape = 90, 70  # nrow, ncol 3 min
bbox = [32, -21.5, 35.5, -17] # W, S, E, N

# read model maps
transform = from_bounds(*bbox, shape[1], shape[0])
nodata=-9999
da_lst = []
nlayers = {'nextxy': 2, 'lonlat': 2}
dtypes = {'nextxy': 'i4'}
for name in ['nextxy', 'lonlat', 'uparea']: #, 'rivhgt', 'rivwth_grwl' 'rivdph1_hc27_hp30', 'rivwth_est']:
    shape0 = (nlayers[name], shape[0], shape[1]) if name in nlayers else shape
    dtype = dtypes.get(name, 'f4')
    da = hydromt.raster.RasterDataArray.from_numpy(
        data = np.fromfile(join(mdir, f'{name}.bin'), dtype).reshape((shape0)),
        transform=transform, 
        nodata=nodata,
    )
    da.name = name
    da_lst.append(da)
ds = xr.merge(da_lst)#.rename({'rivhgt': 'rivdph', 'rivwth_grwl': 'rivwth'})
ds.raster.set_crs(4326)
ds['idx'] = xr.DataArray(data=np.arange(ds.raster.size, dtype=int).reshape(ds.raster.shape), dims=ds.raster.dims)
cmf_mask = ds['uparea'] != ds['uparea'].raster.nodata
ds['uparea'] = (ds['uparea']/1e6).where(cmf_mask, ds['uparea'].raster.nodata) # km2

# initiate flow dir object
flw = hydromt.flw.flwdir_from_da(ds['nextxy'], ftype='nextxy')

In [ ]:
ds['uparea'].raster.set_nodata(nodata)
ds['uparea'].raster.to_raster(join(mdir, 'uparea.tif'))

In [ ]:
# get CaMa-flood outlet locations and convert to point geometry
xs, ys = ds['lonlat'].raster.mask_nodata().values
gdf_out = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(xs.ravel(),ys.ravel(), crs=4326),
    data = {name: ds[name].values.ravel() for name in ds.data_vars if ds[name].ndim == 2}
)
gdf_out = gdf_out[~gdf_out.geometry.is_empty].set_index('idx')  # drop invalid  points

In [ ]:
# get lin 2019 river width and bankfull Q data
from hydromt import DataCatalog
data_cat = DataCatalog(deltares_data=True)
bbox1 = np.nanmin(xs), np.nanmin(ys), np.nanmax(xs), np.nanmax(ys)
print(bbox1)

gdf_rivdata = data_cat.get_geodataframe('rivers_lin2019_v1', bbox=bbox1, buffer=1000) # buffer [m]

In [ ]:
# merge lin river with data with CaMa-Flood data
from hydromt.gis_utils import nearest_merge
cols = ["rivwth", "qbankfull"]
gdf_out1 = nearest_merge(gdf_out, gdf_rivdata, columns=cols, max_dist=2e2)
for col in cols:
    data = np.full(ds.raster.shape, nodata, dtype=float)
    data.flat[gdf_out1.index.values] = gdf_out1[col].fillna(nodata).values
    # ds[col] = xr.DataArray(flw.fillnodata(data, nodata), dims=ds.raster.dims)
    ds[col] = xr.DataArray(data, dims=ds.raster.dims)
    ds[col].raster.set_nodata(nodata)

In [ ]:
# derive river width 
min_rivwth = 5

ds['qbankfull'] = np.maximum(ds['qbankfull'], 0).where(cmf_mask, nodata)
ds['rivwth'] = np.maximum(ds['rivwth'], min_rivwth).where(cmf_mask, nodata)
ds['qbankfull'].values.astype(np.float32).tofile(join(mdir, f'qbankfull.bin'))
ds['rivwth'].values.astype(np.float32).tofile(join(mdir, f'rivwth_lin.bin'))

In [ ]:
# derive river depth
min_rivdph = 1
hc, hp = 0.27, 0.25
ds['rivdph'] = np.maximum(hc * ds['qbankfull']**hp, min_rivdph).where(cmf_mask, nodata)
ds['rivdph'].values.astype(np.float32).tofile(join(mdir, f'rivdph_hc{hc}_hp{hp}.bin'))

In [ ]:
# get CaMa-Flood flow directions and vectorize streams for visualization
feats = flw.vectorize(
    xs=xs, ys=ys, **{name: ds[name].values for name in ds.data_vars if (ds[name].ndim == 2 and name != 'idx')}
)
gdf_riv0 = gpd.GeoDataFrame.from_features(feats, crs=4326).set_index('idx')

gdf_riv0.explore('rivwth', vmin=min_rivwth, vmax=500)


In [ ]:
# (ds['rivdph'] - ds['rivdph1_hc27_hp30']).where(cmf_mask).plot(vmin=-1, vmax=1, cmap='coolwarm')
(ds['rivwth'] - ds['rivwth_est']).where(cmf_mask).plot(vmin=-50, vmax=50, cmap='coolwarm')

In [ ]:
ds_base = data_cat.get_rasterdataset('merit_hydro', bbox=bbox1, buffer=5)  #buffer [cells]


In [ ]:
flw_d8 = hydromt.flw.flwdir_from_da(ds_base['flwdir'], mask=False)
_ = flw_d8.main_upstream(ds_base['uparea'].values)

gdf_out['idxs_out'] = flw_d8.index(*ds_flat['lonlat'].isel(index=gdf_out.index))
mask = np.zeros(flw_d8.shape, dtype=bool).ravel()
for idxs in flw_d8.path(idxs = gdf_out['idxs_out'])[0]:
    mask[idxs] = True
mask = mask.reshape(flw_d8.shape)

feats = flw_d8.streams(
    mask=mask,
    elevtn=ds_base['elevtn'].values,
    uparea=ds_base['uparea'].values,
    direction='down',
    max_len=5e3
)
gdf_stream = gpd.GeoDataFrame.from_features(feats, crs=ds_base.raster.crs)


In [ ]:
#
from hydromt.workflows.rivers import river_width, river_depth

da_rivmask = data_cat.get_rasterdataset('grwl_mask', bbox=bbox1)#.raster.reproject_like(ds_base, method='max')
da_rivmask = np.logical_and(da_rivmask!=255,da_rivmask!=0)

gdf_rivdata = data_cat.get_geodataframe('rivers_lin2019_v1', bbox=bbox1, buffer=1000) # buffer [m]

gdf_stream['rivwth'] = river_width(gdf_stream, da_rivmask=da_rivmask)


In [ ]:
from hydromt_sfincs.workflows import get_river_bathymetry
gdf_riv = get_river_bathymetry(
    ds_base,
    flwdir=flw_d8,
    gdf_riv=gdf_rivdata,
    river_upa = 25,
    river_len = 1e3,
    segment_length=5e3,
    rivdph_method='powlaw',
    hp=0.3,
    hc=0.27,
    rivwth_method='geom'
)


In [ ]:
gdf_stream.explore('rivwth', vmin=5, vmax=300, cmap='Blues')

In [ ]:
gdf_stream.to_file('cmf_streams.gpkg', driver='GPKG')

In [ ]:

mdir1 = r'../../3_models/SFINCS'
runs = [path for path in glob.glob(join(mdir1, '*')) if isdir(path)]
runs

In [ ]:

from hydromt.gis_utils import nearest_merge
cols = ['rivdph', 'rivwth', 'uparea']

for root in runs[2:]:
    fn0 = join(root, 'gis', 'rivers.geojson')
    gdf_riv = gpd.read_file(fn0)
    gdf_out1 = nearest_merge(gdf_out.to_crs(gdf_riv.crs), gdf_riv[['geometry']+cols], max_dist=1e3)
    valid = np.abs(1-gdf_out1['uparea0']/gdf_out1['uparea'])<0.1
    for col in cols:
        vals = gdf_out1.loc[gdf_riv0.idx, col].values
        vals[~valid.loc[gdf_riv0.idx]] = np.nan
        gdf_riv0[col] = np.where(~np.isnan(vals), vals, gdf_riv0[col])
    # gdf_riv0['distance_right'] = gdf_out1.loc[gdf_riv0.idx, 'distance_right'].values
    # gdf_riv0.explore('distance_right', vmax=1e2)#('rivdph')
    for dvar in ['rivdph', 'rivwth']:
        data = ds[dvar].values.copy()
        data.flat[gdf_riv0.idx.values] = gdf_riv0[dvar].values
        data.tofile(join(mdir, f'{dvar}_{basename(root)[3:]}.bin'))
    break